## read data from Excel file

In [110]:
import pandas as pd

# Specify the path to your Excel file
file_path_train = 'snappfood-data/train.csv'
file_path_tst = 'snappfood-data/test.csv'


# Read the dataset from the Excel file
data_train = pd.read_csv(file_path_train ,  sep='; , .')
data_tst = pd.read_csv(file_path_tst, sep='; , .')

# Display the first few rows of the dataframe
print(data_train.head())

C:\Users\RABIN-IT\AppData\Local\Temp\ipykernel_14980\2322229779.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data_train = pd.read_csv(file_path_train ,  sep='; , .')


                            comment\tlabel\tlabel_id
0  0\tواقعا حیف وقت که بنویسم سرویس دهیتون شده اف...
1  1\tقرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از...
2  2\tقیمت این مدل اصلا با کیفیتش سازگاری نداره، ...
3  3\tعالللی بود همه چه درست و به اندازه و کیفیت ...
4         4\tشیرینی وانیلی فقط یک مدل بود.\tHAPPY\t0


C:\Users\RABIN-IT\AppData\Local\Temp\ipykernel_14980\2322229779.py:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data_tst = pd.read_csv(file_path_tst, sep='; , .')


## remove empty and duplicate input

In [111]:
df_remove_train = data_train.dropna().drop_duplicates()
df_remove_tst = data_tst.dropna().drop_duplicates()


In [112]:
len(df_remove_train)

56700

## create input and output matrix 

In [113]:
input_train = [[] for i in range(len(df_remove_train))]
               
output_train = [0 for i in range(len(df_remove_train))]

for i in range(0, len(df_remove_train)):
    spilt = df_remove_train[i:i+1].to_string().split('\\')
    input_train[i] = spilt[3]
    output_train[i] = spilt[5][1]
    
    
input_tst = [[] for i in range(len(df_remove_tst))]
               
output_tst = [0 for i in range(len(df_remove_tst))]

for j in range(0, len(df_remove_tst)):
    spilt_tst = df_remove_tst[j:j+1].to_string().split('\\')
    input_tst[j] = spilt_tst[3]
    output_tst[j] = spilt_tst[5][1]
    

## check the values of input and output

In [114]:
# Create new lists that filter out the unwanted elements
filtered_input_train = [input_train[i] for i in range(len(input_train)) if input_train[i] != '' and output_train[i] != 'S']
filtered_output_train = [output_train[i] for i in range(len(output_train)) if input_train[i] != '' and output_train[i] != 'S']

# Update the original lists
input_train = filtered_input_train
output_train = filtered_output_train


# Test the input and output
filtered_input_tst = [input_tst[i] for i in range(len(input_tst)) if input_tst[i] != '' and output_tst[i] != 'S']
filtered_output_tst = [output_tst[i] for i in range(len(input_tst)) if input_tst[i] != '' and output_tst[i] != 'S']

# Update the original lists
input_tst = filtered_input_tst
output_tst = filtered_output_tst
        


    

In [115]:
output_train = [int(x) for x in output_train if x.isdigit()]
output_tst = [int(x) for x in output_tst if x.isdigit()]




In [116]:
print(output_train[0:10])

[1, 0, 1, 0, 0, 1, 0, 1, 0, 1]


## tokenize input_train and tst

In [117]:
from transformers import BertTokenizer
tokenizer_train = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# Tokenize with padding and truncation
encoded_comments_trian = tokenizer_train(
    input_train,
    padding=True,           # Pads to the longest sequence in the batch
    truncation=True,        # Truncates any input longer than max_length
    max_length=64,         # Adjust max_length based on your requirements
    return_tensors='pt'    # Return PyTorch tensors (use 'tf' for TensorFlow)
)



In [118]:
tokenizer_tst = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# Tokenize with padding and truncation
encoded_comments_tst = tokenizer_tst(
    input_tst,
    padding=True,           # Pads to the longest sequence in the batch
    truncation=True,        # Truncates any input longer than max_length
    max_length=64,         # Adjust max_length based on your requirements
    return_tensors='pt'    # Return PyTorch tensors (use 'tf' for TensorFlow)
)



In [119]:
output_train.append(0)

# Data Loader

In [120]:
import torch
from torch.utils.data import TensorDataset, DataLoader



# and labels are the corresponding labels for each input.
# Assume input_ids and attention_mask are the tokenized input tensors, 
input_ids_train = encoded_comments_trian['input_ids']  # Example tokenized input
attention_mask_train = encoded_comments_trian['attention_mask']  # Attention masks
 

# Convert lists to PyTorch tensors
input_ids_tensor = torch.tensor(input_ids_train)
attention_mask_tensor = torch.tensor(attention_mask_train)
labels_tensor = torch.tensor(output_train)

# Create a TensorDataset
dataset_train = TensorDataset(input_ids_tensor, attention_mask_tensor, labels_tensor)

# Create DataLoader
dataloader_train = DataLoader(dataset_train, batch_size=2, shuffle=True)




C:\Users\RABIN-IT\AppData\Local\Temp\ipykernel_14980\313625587.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids_tensor = torch.tensor(input_ids_train)
C:\Users\RABIN-IT\AppData\Local\Temp\ipykernel_14980\313625587.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask_tensor = torch.tensor(attention_mask_train)


In [121]:
# and labels are the corresponding labels for each input.
# Assume input_ids and attention_mask are the tokenized input tensors, 
input_ids_tst = encoded_comments_tst['input_ids']  # Example tokenized input
attention_mask_tst = encoded_comments_tst['attention_mask']  # Attention masks
 

# Convert lists to PyTorch tensors
input_ids_tensor = torch.tensor(input_ids_tst)
attention_mask_tensor = torch.tensor(attention_mask_tst)
labels_tensor = torch.tensor(output_tst)

# Create a TensorDataset
dataset_tst = TensorDataset(input_ids_tensor, attention_mask_tensor, labels_tensor)

# Create DataLoader
dataloader_tst = DataLoader(dataset_tst, batch_size=2, shuffle=True)




C:\Users\RABIN-IT\AppData\Local\Temp\ipykernel_14980\2776536946.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids_tensor = torch.tensor(input_ids_tst)
C:\Users\RABIN-IT\AppData\Local\Temp\ipykernel_14980\2776536946.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask_tensor = torch.tensor(attention_mask_tst)
